This notebook is used to create the dataframe with all Dutch municipalities. The CBS Wijk- en Buurtkaart dataset is used as source.

In [2]:
# import packages

import numpy as np 
import pandas as pd  # provides interface for interacting with tabular data
import geopandas as gpd  # combines the capabilities of pandas and shapely for geospatial operations
from shapely.geometry import Point, Polygon, MultiPolygon  # for manipulating text data into geospatial shapes
from shapely import wkt  # stands for "well known text," allows for interchange across GIS programs
import rtree  # supports geospatial join

# import seaborn as sns
import matplotlib. pyplot as plt # for plotting graphs
import matplotlib.mlab as mlab # 
import matplotlib
plt.style.use('ggplot') # basic, but functional, plotstyle,
from matplotlib.pyplot import figure
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)

In [2]:
# reading data

# CBS dataset
municipalities_input = gpd.read_file("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Input/Gemeente/gemeente_2017_v3.shp")

# Municipalities and MR-stations 
municipalities_MR = pd.read_excel("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/Gemeente_excelGasunie/municipalities.xls")

# List of unique MR-stations
MR_stations = pd.read_excel("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/Gemeente_excelGasunie/municipalities.xls", sheet_name = "Unique MR")

In [3]:
# cleaning data

# selecting the columns that need to be included
municipalities_1 = municipalities_input[['GM_NAAM','GM_CODE','WATER','AANT_INW','AANTAL_HH','geometry']]

# remove the non-sensible entries that are water
municipalities_2 = municipalities_1.loc[municipalities_1['WATER'] == 'NEE']




In [4]:
# transforming data

In [5]:
# creating a municipality distance matrix

# function that takes two series of names and a df source and then computes the distances between all its entries
# it returns a distance dataframe
def dist_geopandas_2(names_i, names_j, source):
    df = pd.DataFrame(index=names_i, columns=names_j) # create an empty df with the mp names as index and column names
    df = df.fillna(0) # fill it empty
    for i in names_i: # iterate over names
        for j in names_j: # iterate over names
            mp_i = gpd.GeoDataFrame(source[['GM_NAAM', 'centroid']][source.GM_NAAM == i]) # get geopandas dataframe for entry i
            mp_j = gpd.GeoDataFrame(source[['GM_NAAM', 'centroid']][source.GM_NAAM == j]) # get geopandas dataframe for entry j
            mp_i = mp_i.rename(columns={'centroid':'geometry'}).reset_index(drop=True) # rename the centroid columnn to geometry so that the distance function can computer
            mp_j = mp_j.rename(columns={'centroid':'geometry'}).reset_index(drop=True) # rename the centroid columnn to geometry so that the distance function can compute
            
            dist = mp_i.distance(mp_j) # calculate the distances
            
            df.at[i,j] = dist # add distance to dataframe
            
    return df

In [6]:
# function that takes the municipalities of interest, the variable that is used as a mass proxy, and the source dataframes
# then outputs a dataframe containing the gravity outputs for all municipality combinations
def gravitymodel(names_i, names_j, mass_variable, source_dist, source_mass): 
    df = pd.DataFrame(index=names_i, columns=names_j) # create an empty df with names on rows and columns
    df = df.fillna(0)
    for i in names_i:
        for j in names_j:
            mass_i = municipalities_2[municipalities_2.GM_NAAM == i].AANT_INW.item() # extracts the mass variable
            mass_j = municipalities_2[municipalities_2.GM_NAAM == j].AANT_INW.item() # extracts the mass variable
            dist_i_j = df_dist[i][j] # extracts the distance between i and j
            if dist_i_j == 0: # this ifelse statement prevents the info_vol formula from calculating infinity
                info_vol = 0
            else:
                info_vol = (mass_i * mass_j)/dist_i_j #squared????
            
            df.at[i,j] = info_vol # place the info_vol output in the dataframe at the correct place
    return df 

In [7]:
#df_dist = dist_geopandas_2(municipalities_2.GM_NAAM, municipalities_2.GM_NAAM, municipalities_2)

# write the df to a file for speed purposes
# df_dist.to_csv('/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/distance_df.csv')

In [8]:
# df_gravity = gravitymodel(municipalities_2.GM_NAAM, municipalities_2.GM_NAAM, 'AANT_INW', df_dist, municipalities_2)

# write the df to a file for speed purposes
# df_gravity.to_csv('/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/gravity_df.csv')

In [5]:
# add MR-stations columnn to df

municipalities_3 = municipalities_2.merge(municipalities_MR, how = "inner", on = "GM_NAAM").drop(['Unnamed: 0', 'Station 2'], axis = 1)

In [6]:
# create df of unique MR stations



In [7]:
# transforming the projection system to WGS84 to be readable for Netlogo
municipalities_3_WGS84 = municipalities_3.to_crs("EPSG:32632")

In [27]:
# Writing data

# write the df to a file for speed purposes
# df_dist.to_csv('/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/distance_df.csv')

# write the df to a file for speed purposes
# df_gravity.to_csv('/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/gravity_df.csv')

# write the municipalities dataframe to file
#municipalities_3_WGS84.to_file("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/Gemeente/04_08/municipalities.shp")

# write municipalities df to excel file
#municipalities_2_WGS84.GM_NAAM.to_excel("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/Gemeente_excelGasunie/municipalities.xls")

# write unique MR stations to file
#MR_stations.to_csv("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/Gemeente/04_08/MR_stations.csv")

In [4]:
# descriptives used dataset

mp_dataset = gpd.read_file("/Users/jaromirbogdanovski/Documents/Studie/EPA/Master Thesis/Igor Nikolic/Coding/Data/Output/Gemeente/04_08/municipalities.shp")

In [6]:
mp_dataset

,GM_NAAM,GM_CODE,WATER,AANT_INW,AANTAL_HH,Station 1,geometry
0,Appingedam,GM0003,NEE,11971,5600,Scheemda,"POLYGON ((353839.370 5909783.240, 353842.934 5..."
1,Bedum,GM0005,NEE,10479,4399,Oostum,"POLYGON ((337192.538 5907060.071, 337194.266 5..."
2,Bellingwedde,GM0007,NEE,8919,3989,Scheemda,"POLYGON ((367873.097 5886598.533, 367869.490 5..."
3,Ten Boer,GM0009,NEE,7288,2958,Oostum,"POLYGON ((349469.059 5909379.570, 349470.098 5..."
4,Delfzijl,GM0010,NEE,24965,11527,Scheemda,"MULTIPOLYGON (((373511.642 5909827.229, 373511..."
...,...,...,...,...,...,...,...
383,Gooise Meren,GM1942,NEE,56935,25494,Hilversum,"MULTIPOLYGON (((232988.316 5806588.703, 232988..."
384,Berg en Dal,GM1945,NEE,34764,15494,Ewijk,"POLYGON ((286078.690 5747866.096, 286076.136 5..."
385,Meierijstad,GM1948,NEE,79864,33611,Boxtel,"POLYGON ((250051.381 5725096.529, 250050.949 5..."
386,Montferland,GM1955,NEE,35316,15161,Angerlo,"POLYGON ((304307.023 5754994.503, 304306.372 5..."


In [7]:
df_latex1 = mp_dataset[['AANTAL_HH']].describe()

In [8]:
print(df_latex1.to_latex())

\begin{tabular}{lr}
\toprule
{} &      AANTAL\_HH \\
\midrule
count &     388.000000 \\
mean  &   20087.822165 \\
std   &   35522.701497 \\
min   &     509.000000 \\
25\%   &    7465.250000 \\
50\%   &   11291.500000 \\
75\%   &   19398.000000 \\
max   &  462329.000000 \\
\bottomrule
\end{tabular}

